In [1]:
##################################################################################################################
############################## LLM ENGINEERING: Jordi Ortega - IA2 ###############################################
##################################################################################################################


# GenAI Project Lifecycle. Pasos aplicados en este notebook:
# 1. DefiniCión, modelo y adaptación (Entrenamos al modelo para que tenga conocimiento sobre salud mental)
# 2. Conclusiones

In [2]:
##################################################################################################################
############################## DEFINICIÓN, MODELO y ADAPTACIÓN ###################################################
##################################################################################################################

In [3]:
from huggingface_hub import login

# Usamos token de perfil propio para descarga del modelo en HF
login(token="hf_RDUAAxxxxxxxxxxxxxxxxxsnCvKANYKB")

In [4]:
!pip install -qU bitsandbytes datasets accelerate loralib peft transformers trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.6/336.6 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 118.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requi

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Cargamos el modelo
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("JordiOrtega/distilgpt2")
model = AutoModelForCausalLM.from_pretrained("JordiOrtega/distilgpt2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/507 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [6]:
tokenizer.pad_token = tokenizer.eos_token

In [7]:
model.resize_token_embeddings(len(tokenizer))

Embedding(50257, 768)

In [8]:
# Descargamos el dataset y lo guardarmos en Drive
from datasets import load_dataset
import os

dataset_name = "Amod/mental_health_counseling_conversations"
dataset = load_dataset(dataset_name)

drive_dataset_path = "/content/drive/MyDrive/datasets"
os.makedirs(drive_dataset_path, exist_ok=True)
dataset.save_to_disk(drive_dataset_path)
print("Dataset guardado en Google Drive.")

README.md:   0%|          | 0.00/2.82k [00:00<?, ?B/s]

combined_dataset.json:   0%|          | 0.00/4.79M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3512 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3512 [00:00<?, ? examples/s]

Dataset guardado en Google Drive.


In [9]:
# Confirmamos la estructura del dataset
dataset['train'][0]

{'Context': "I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless to everyone?",
 'Response': "If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why you're not worthless, then go back to the same crowd and be knocked down again.There are many inspirational messages you can find in social media. \xa0Maybe read some of the ones which state that no person is worthless, and that everyone has a good purpose to their life.Also, since our culture is so saturated with the belief that if someone doesn't feel good about themselves that this is someh

In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Context', 'Response'],
        num_rows: 3512
    })
})

In [11]:
# Filtramos los registros en 'train' donde 'output' no es None ni vacío
dataset['train'] = dataset['train'].filter(lambda row: row['Response'] and row['Response'].strip() != "")

print(f"Tamaño después del filtro: {len(dataset['train'])}")


Filter:   0%|          | 0/3512 [00:00<?, ? examples/s]

Tamaño después del filtro: 3508


In [12]:
# Dividimos el dataset en train y test
dataset_train_test = dataset["train"].train_test_split(test_size=0.2)
dataset_train_test

DatasetDict({
    train: Dataset({
        features: ['Context', 'Response'],
        num_rows: 2806
    })
    test: Dataset({
        features: ['Context', 'Response'],
        num_rows: 702
    })
})

In [13]:
# Dividimos el dataset test en validación
dataset_val_test = dataset_train_test["test"].train_test_split(test_size=0.5)
dataset_val_test

DatasetDict({
    train: Dataset({
        features: ['Context', 'Response'],
        num_rows: 351
    })
    test: Dataset({
        features: ['Context', 'Response'],
        num_rows: 351
    })
})

In [14]:
from datasets import DatasetDict

# Estructura del dataset preparada para el modelo
split_dataset = DatasetDict({
    "train" : dataset_train_test["train"],
    "val" : dataset_val_test["train"],
    "test" : dataset_val_test["test"]
})
split_dataset

DatasetDict({
    train: Dataset({
        features: ['Context', 'Response'],
        num_rows: 2806
    })
    val: Dataset({
        features: ['Context', 'Response'],
        num_rows: 351
    })
    test: Dataset({
        features: ['Context', 'Response'],
        num_rows: 351
    })
})

In [15]:
# Creamos los dos templates de prompting, el de entreno y el de inferencia
SYSTEM_MESSAGE= "You are a personal assistant expert in mental health issues that responds concisely based on the given context in order to help people. You receive a context and you must provide a concrete and helpful answer."

PROMPT_TEMPLATE = """\
{bos_token}
{system_message}

### Context:
{Context}

### Response:
{Response}{eos_token}
"""

def create_prompt(sample):
  full_prompt = PROMPT_TEMPLATE.format(
      bos_token = "<|startoftext|>",
      eos_token = "<|endoftext|>",
      system_message = SYSTEM_MESSAGE,
      Context = sample["Context"],
      Response = sample["Response"]
  )

  return {"text" : full_prompt}

In [16]:
INFERENCE_PROMPT_TEMPLATE = """\
{bos_token}
{system_message}

### Context:
{Context}

### Response:
{eos_token}
"""

def create_prompt_and_response(sample):
  full_prompt = INFERENCE_PROMPT_TEMPLATE.format(
      bos_token = "<|startoftext|>",
      eos_token = "<|endoftext|>",
      system_message = SYSTEM_MESSAGE,
      Context = sample["Context"],
  )

  ground_truth = sample["Response"]

  return {"full_prompt" : full_prompt, "ground_truth" : ground_truth}

In [17]:
# Primer registro del dataset
split_dataset["train"][0]

{'Context': 'We are the legal guardianship of our grandson. He has been acting out, and he wants to talk to his mom that has not talked to him since before last year. How are we to handle this?',
 'Response': "Hi Boone, This depends a lot on his age and the situation. The younger the child is, in general, the more I would see this as your decision, not his. A child of any age should be listened to carefully about these matters, but just because a child has unresolved issues about a parent doesn't necessarily mean that having contact is a good solution.\xa0Some questions... Was the parent harmful to the child? Is the parent likely to be harmful to the child? How likely is it to be a positive experience? Do you have any contact with the parent? You may want to speak with the parent first to determine whether they are in a stable enough place to allow this to happen. \xa0I would start by asking the child more about what is on his mind. What does he want to get out of a visit with the pare

In [18]:
# Aplicamos la función al primer registro del dataset
create_prompt(split_dataset["train"][0])

{'text': "<|startoftext|>\nYou are a personal assistant expert in mental health issues that responds concisely based on the given context in order to help people. You receive a context and you must provide a concrete and helpful answer.\n\n### Context:\nWe are the legal guardianship of our grandson. He has been acting out, and he wants to talk to his mom that has not talked to him since before last year. How are we to handle this?\n\n### Response:\nHi Boone, This depends a lot on his age and the situation. The younger the child is, in general, the more I would see this as your decision, not his. A child of any age should be listened to carefully about these matters, but just because a child has unresolved issues about a parent doesn't necessarily mean that having contact is a good solution.\xa0Some questions... Was the parent harmful to the child? Is the parent likely to be harmful to the child? How likely is it to be a positive experience? Do you have any contact with the parent? You 

In [19]:
# Pasamos por el template todo el dataset
split_dataset = split_dataset.map(create_prompt)

Map:   0%|          | 0/2806 [00:00<?, ? examples/s]

Map:   0%|          | 0/351 [00:00<?, ? examples/s]

Map:   0%|          | 0/351 [00:00<?, ? examples/s]

In [20]:
from transformers import pipeline, set_seed, GenerationConfig

# Creamos el pipeline para testear el modelo base (posteriormente lo haremos con el modelo finetuneado)
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

set_seed(42)

def generate_sample(sample):
  prompt_package = create_prompt_and_response(sample)

  generation_config = GenerationConfig(
      max_new_tokens=100,
      do_sample=True,
      top_k=50,
      temperature=1e-4,
      eos_token_id=model.config.eos_token_id,
  )

  generation = generator(prompt_package["full_prompt"], generation_config=generation_config)
  print("---------------")
  print("Question:")
  print(sample["Context"])
  print("---------------")
  print("Dataset Response")
  print(prompt_package["ground_truth"])
  print("---------------")
  print("Model Response:")
  print(generation[0]["generated_text"].replace(prompt_package["full_prompt"], ""))


Device set to use cuda:0


In [21]:
generate_sample(split_dataset["test"][15])
# Comprobamos que el modelo base no tiene expertise en salud mental.

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------------
Question:
How does a counselor decide when to end counseling sessions or to terminate working with a client?
---------------
Dataset Response
To be able to identify a clear ending to a counseling relationship, we must have a clear understanding of the goals and limitations of treatment.  Usually, during the initial evaluation, I identify my client’s short term and long term therapy goals.  As therapy progress, we verify goals on a monthly to a quarterly basis, depending on the frequency of the appointments.  Once the therapy goals have been met, there is a closing session, the counseling relationship is ended, and the client can stop attending sessions.  In some cases, the client can also establish new goals and determine if I am a good fit for their therapy needs or if they need a new provider, in which case I provide multiple referrals.The counseling relationship could also end due to other factors like noncompliance= when a client is not committed to the agreed treat

In [32]:
from transformers import TrainingArguments

# Definimos los argumentos del entreno
from transformers import TrainingArguments
from trl import SFTTrainer

training_args = TrainingArguments(
 per_device_train_batch_size=4,
 gradient_accumulation_steps=4,
 gradient_checkpointing =True,
 max_grad_norm= 0.3,
 max_steps=500,
 learning_rate=2e-4,
 save_total_limit=3,
 logging_steps=10,
 output_dir="finetuned_mental_health_distilgpt2",
 optim="paged_adamw_32bit",
 lr_scheduler_type="cosine",
 evaluation_strategy="steps",
 eval_steps=50,
 warmup_ratio=0.05,
 report_to="none",
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [33]:
from trl import SFTTrainer

# Facilitamos la configuración a SFTTrainer (supervised finetuning)
trainer = SFTTrainer(
 model,
 train_dataset=split_dataset["train"],
 eval_dataset=split_dataset["val"],
 tokenizer=tokenizer,
 args=training_args
)

<ipython-input-33-b6571abee755>:4: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map:   0%|          | 0/2806 [00:00<?, ? examples/s]

In [34]:
# Entrenamos
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss,Validation Loss
50,2.780900,2.664924
100,2.632100,2.525020
150,2.590400,2.453177
200,2.387800,2.404581
250,2.346900,2.354420
300,2.321300,2.326340
350,2.278000,2.295965
400,2.217800,2.288771
450,2.207600,2.283498
500,2.225000,2.283103


TrainOutput(global_step=500, training_loss=2.4286077461242677, metrics={'train_runtime': 1108.4749, 'train_samples_per_second': 7.217, 'train_steps_per_second': 0.451, 'total_flos': 1099947044044800.0, 'train_loss': 2.4286077461242677, 'epoch': 2.8433048433048436})

In [35]:
trainer.save_model()

In [36]:
distilbert_mental_health_distilgpt2_finetuned = AutoModelForCausalLM.from_pretrained("finetuned_mental_health_distilgpt2")

In [38]:
generator = pipeline('text-generation', model=distilbert_mental_health_distilgpt2_finetuned, tokenizer=tokenizer)

Device set to use cuda:0


In [42]:
# Testeamos ahora sí con el modelo finetuneado.
generate_sample(split_dataset["test"][20])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------------
Question:
Every winter I find myself getting sad because of the weather.  How can I fight this?
---------------
Dataset Response
Seasonal Affective Disorder (S.A.D.) is a term that reflects how many people are affected by the changing seasons, especially fall to winter.  Everyone suffers with some form of this (lessened activity levels, increased isolation, etc.) while some find that this time of year can put them into a deeper depression. If you have noticed that this happens frequently, there are some ways you can definitely help yourself going forward:1. Attend therapy to learn strategies and tools to help you to manage your mood.  It's important to stay within the therapy until you feel you have mastered these tools. 2. Push yourself to interact more with your social groups and other positive activities. It's easy to go out and spend the day outside in the summer months, when the temperature is warm and the sun shines for long periods of the day, but it seems harder

In [ ]:
# Conclusiones:
# Como en el primer fine-tuning los resultados mejoran sobre el modelo base, pero es evidente que el modelo dista de ser experto en enfermedades mentales.
# Conseguimos un modelo con empatía pero que no da respuestas concretas a los inputs.
# La evaluación del modelo, más allá del resultado del prompt contra el propio dataset de entreno, se ha realizado contra gpt-4o (se adjunta png de la interfaz) y confirma la distancia entre ambos modelos.
# Las dos principales acciones a realizar serían:
# 1. Ampliar el dataset
# 2. Valorar la capacidad de un modelo tan pequeño
# Además, se listan siguientes pasos a seguir para mejorar el modelo y evitar las "repeticiones de texto" que parece hacer una vez ha dado una primera respuesta genérica pero coherente.
# 1. El dataset no parece tener contenido repetitivo, pero convendría ampliarlo para producir un contenido más rico
# 2. Probar otros parámetros en el prompt como temeprature, top_k y max_new_token.
# 3. Probar otros hiperparámetros en el 'training arguments' como el learning rate y el número de épocas.
# 4. Añadir en la inferencia el parámetro repetition_penalty para evitar repeticiones de palabras en el output del prompt

In [40]:
# Limpiamos la parte repetitiva y redudante de la respuesta
def remove_redundant_phrases(response):
    words = response.split()
    unique_words = []
    for word in words:
        if word not in unique_words:
            unique_words.append(word)
    return " ".join(unique_words)

# Ejemplo anterior
model_response = "Hello, and thank you for your question. I am so sorry that you are feeling this way. I am so sorry that you are feeling this way. I am so sorry that you are feeling this way. I am sorry that you are feeling this way. I am sorry that you are feeling this way. I am sorry that you are feeling this way. I am sorry that you are feeling this way. I am sorry that you are feeling this way. I am sorry that you are feeling this"

cleaned_response = remove_redundant_phrases(model_response)
print(cleaned_response)


Hello, and thank you for your question. I am so sorry that are feeling this way.
